In [62]:
import pandas as pd
from scipy.spatial import distance
from sklearn.preprocessing import OrdinalEncoder
from sklearn.naive_bayes import MultinomialNB
import sys
sys.path.insert(0, '../../Code/')


In [63]:
data = {
    'Ambiente': ['Soleado', 'Soleado', 'Nublado', 'Lluvioso', 'Lluvioso', 'Lluvioso', 'Nublado', 
                'Soleado', 'Soleado', 'Lluvioso', 'Soleado', 'Nublado', 'Nublado', 'Lluvioso'],
    'Temperatura': ['Alta', 'Alta', 'Alta', 'Media', 'Baja', 'Baja', 'Baja', 
                    'Media', 'Baja', 'Media', 'Media', 'Media', 'Alta', 'Media'],
    'Humedad': ['Alta', 'Alta', 'Alta', 'Alta', 'Normal', 'Normal', 'Normal', 
                'Alta', 'Normal', 'Normal', 'Normal', 'Alta', 'Normal', 'Alta'],
    'Viento': ['Leve', 'Fuerte', 'Leve', 'Leve', 'Fuerte', 'Fuerte', 'Leve', 
               'Leve', 'Leve', 'Leve', 'Fuerte', 'Fuerte', 'Leve', 'Fuerte'],
    'JugarTenis': ['No', 'No', 'Sí', 'No', 'No', 'No', 'Sí', 
                   'Sí', 'Sí', 'No', 'Sí', 'Sí', 'Sí', 'No']
}

df = pd.DataFrame(data)

In [64]:
df

,Ambiente,Temperatura,Humedad,Viento,JugarTenis
0,Soleado,Alta,Alta,Leve,No
1,Soleado,Alta,Alta,Fuerte,No
2,Nublado,Alta,Alta,Leve,Sí
3,Lluvioso,Media,Alta,Leve,No
4,Lluvioso,Baja,Normal,Fuerte,No
5,Lluvioso,Baja,Normal,Fuerte,No
6,Nublado,Baja,Normal,Leve,Sí
7,Soleado,Media,Alta,Leve,Sí
8,Soleado,Baja,Normal,Leve,Sí
9,Lluvioso,Media,Normal,Leve,No


In [65]:
nval = pd.DataFrame({"Ambiente":["Soleado"],
                     'Temperatura':["Baja"],
                     'Humedad':["Alta"],
                     'Viento':["Fuerte"]})

In [66]:
nval

,Ambiente,Temperatura,Humedad,Viento
0,Soleado,Baja,Alta,Fuerte


In [67]:
Xy = df

In [68]:
for cl in Xy["JugarTenis"].unique():
    print(Xy[Xy["JugarTenis"]==cl].shape[0], cl)

7 No
7 Sí


In [69]:
#NO = 7
#SI = 7
for c in Xy.columns.drop("JugarTenis"):
    for cla in Xy[c].unique():
        for cl in Xy["JugarTenis"].unique():
            #cl si o no
            cant = Xy[(Xy["JugarTenis"]==cl) & (Xy[c]==cla)].shape[0]
            cantClase = Xy[Xy["JugarTenis"]==cl].shape[0]
            print("{} {} {} {:.3f}".format(c, cla, cl,  cant/cantClase))

Ambiente Soleado No 0.286
Ambiente Soleado Sí 0.429
Ambiente Nublado No 0.000
Ambiente Nublado Sí 0.571
Ambiente Lluvioso No 0.714
Ambiente Lluvioso Sí 0.000
Temperatura Alta No 0.286
Temperatura Alta Sí 0.286
Temperatura Media No 0.429
Temperatura Media Sí 0.429
Temperatura Baja No 0.286
Temperatura Baja Sí 0.286
Humedad Alta No 0.571
Humedad Alta Sí 0.429
Humedad Normal No 0.429
Humedad Normal Sí 0.571
Viento Leve No 0.429
Viento Leve Sí 0.714
Viento Fuerte No 0.571
Viento Fuerte Sí 0.286


In [70]:
#Soleado	Baja	Alta	Fuerte	Sí
noSeJuega = 0.5*(0.285 * 0.285 * 0.571 * 0.571)
siSeJuega = 0.5*(0.428 * 0.285 * 0.428 * 0.285)
print("No se juega {:.5f}, si se juega {:.5f}".format(noSeJuega, siSeJuega))
print("No se juega {:.2f}, si se juega {:.2f}".format(noSeJuega/(noSeJuega+siSeJuega), siSeJuega/(siSeJuega+noSeJuega)))



No se juega 0.01324, si se juega 0.00744
No se juega 0.64, si se juega 0.36


### Predicción utilizando OrdinalEncoder y MultinomialNB()

In [71]:
col_predict = ['Ambiente', 'Temperatura', 'Humedad', 'Viento']
col_target = 'JugarTenis'
X = df.loc[:, col_predict]
y = df.loc[:, col_target]
encoders = {}
for c in col_predict:
    oe = OrdinalEncoder()
    oe.fit(X.loc[:,[c]])
    X.loc[:,[c]] = oe.transform(X.loc[:,[c]])
    nval.loc[:,[c]] = oe.transform(nval.loc[:,[c]])
    encoders[c] = oe



In [72]:
for c in col_predict:
    X[c] = X[c].astype(int)


In [73]:
X

,Ambiente,Temperatura,Humedad,Viento
0,2,0,0,1
1,2,0,0,0
2,1,0,0,1
3,0,2,0,1
4,0,1,1,0
5,0,1,1,0
6,1,1,1,1
7,2,2,0,1
8,2,1,1,1
9,0,2,1,1


In [74]:
model = MultinomialNB()
model.fit(X, y)
model.predict_proba(nval)

array([[0.36630987, 0.63369013]])

In [75]:
prbas = ["{:.3f}".format(i) for i in list(model.predict_proba(nval)[0])]
print("Prediccion {} Probabilidad: {} clases {}".format(model.predict(nval), prbas, model.classes_))

Prediccion ['Sí'] Probabilidad: ['0.366', '0.634'] clases ['No' 'Sí']


----

### LabelEncoder

In [79]:
data = {
    'Ambiente': ['Soleado', 'Soleado', 'Nublado', 'Lluvioso', 'Lluvioso', 'Lluvioso', 'Nublado', 
                'Soleado', 'Soleado', 'Lluvioso', 'Soleado', 'Nublado', 'Nublado', 'Lluvioso'],
    
}
df = pd.DataFrame(data)
le = OrdinalEncoder()
le.fit(df[["Ambiente"]])
df.loc[:,["Ambiente_"]] = le.transform(df[["Ambiente"]])

#### ¿Que clases Obtuvo?

In [80]:
le.inverse_transform([[0], [1], [2]])

array([['Lluvioso'],
       ['Nublado'],
       ['Soleado']], dtype=object)

In [81]:
df.head()

,Ambiente,Ambiente_
0,Soleado,2.0
1,Soleado,2.0
2,Nublado,1.0
3,Lluvioso,0.0
4,Lluvioso,0.0


In [82]:
df.dtypes

Ambiente      object
Ambiente_    float64
dtype: object

#### ¿Como se que categoria es cada número?

In [83]:
le.inverse_transform(df.loc[:, ["Ambiente_"]])

array([['Soleado'],
       ['Soleado'],
       ['Nublado'],
       ['Lluvioso'],
       ['Lluvioso'],
       ['Lluvioso'],
       ['Nublado'],
       ['Soleado'],
       ['Soleado'],
       ['Lluvioso'],
       ['Soleado'],
       ['Nublado'],
       ['Nublado'],
       ['Lluvioso']], dtype=object)